In [1]:
import pandas as pd

In [40]:
df = pd.read_csv('df_for_catboost.csv')

In [41]:
X = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Номер животного', 'Ушная бирка животного', 'Пол', 'Результат отела', 'Дата рождения', 'Событие', 'Дата события', 'Примечание события', 'MOTHER_ID', 'DAYS_OF_TREATMENT'])
y = df['DAYS_OF_TREATMENT']

In [42]:
from catboost import CatBoostRegressor, Pool

cb = CatBoostRegressor(n_estimators=1000,
                       loss_function='RMSE',
                       learning_rate=0.4,
                       depth=3, task_type='CPU',
                       random_state=1,
                       verbose=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)
pool_all = Pool(X, y,
                  cat_features = ['PROTOCOL_STAGE_1', 'PROTOCOL_STAGE_2', 'PROTOCOL_STAGE_3', 'PROTOCOL_STAGE_4', 'PROTOCOL_STAGE_5', 'PROTOCOL_STAGE_6', 'PROTOCOL_STAGE_7', 'PROTOCOL_STAGE_8', 'PROTOCOL_STAGE_9', 'PROTOCOL_STAGE_10', 'PROTOCOL_STAGE_11'])

pool_train = Pool(X_train, y_train,
                  cat_features = ['PROTOCOL_STAGE_1', 'PROTOCOL_STAGE_2', 'PROTOCOL_STAGE_3', 'PROTOCOL_STAGE_4', 'PROTOCOL_STAGE_5', 'PROTOCOL_STAGE_6', 'PROTOCOL_STAGE_7', 'PROTOCOL_STAGE_8', 'PROTOCOL_STAGE_9', 'PROTOCOL_STAGE_10', 'PROTOCOL_STAGE_11'])

pool_test = Pool(X_test, cat_features = ['PROTOCOL_STAGE_1', 'PROTOCOL_STAGE_2', 'PROTOCOL_STAGE_3', 'PROTOCOL_STAGE_4', 'PROTOCOL_STAGE_5', 'PROTOCOL_STAGE_6', 'PROTOCOL_STAGE_7', 'PROTOCOL_STAGE_8', 'PROTOCOL_STAGE_9', 'PROTOCOL_STAGE_10', 'PROTOCOL_STAGE_11'])



In [43]:
cb.fit(pool_train)
y_pred = cb.predict(pool_test)

import numpy as np
from sklearn.metrics import mean_squared_error as mse

cb_rmse = np.sqrt(mse(y_test, y_pred))
print("RMSE in y units:", np.mean(cb_rmse))

RMSE in y units: 14.661907138613712


In [6]:
y.describe()

count    5257.000000
mean       28.398706
std        21.291279
min         1.000000
25%        13.000000
50%        21.000000
75%        37.000000
max       100.000000
Name: DAYS_OF_TREATMENT, dtype: float64

In [7]:
list(zip(cb.get_feature_importance(), X.columns))

[(2.15429323113188, 'Номер лактации'),
 (0.9290790361018125, 'Легкость отела'),
 (6.973886872181285, 'Дней в сухостое предыдущей лактации'),
 (7.932982912121105, 'Дней стельности при событии'),
 (8.417826747049164, 'Номер группы животного'),
 (4.346862680212461, 'Предыдущий номер группы животного'),
 (8.042530706638848, 'Дни доения при событии'),
 (0.9971795235454309, 'IS_COW_ON_HORMONES'),
 (5.937736274758046, 'PROTOCOL_STAGE_1'),
 (2.303814536151736, 'PROTOCOL_STAGE_2'),
 (0.7353581987392237, 'PROTOCOL_STAGE_3'),
 (0.3358972279254993, 'PROTOCOL_STAGE_4'),
 (0.16166844994582677, 'PROTOCOL_STAGE_5'),
 (0.11890150328794685, 'PROTOCOL_STAGE_6'),
 (0.03402405767573168, 'PROTOCOL_STAGE_7'),
 (0.05062929937935773, 'PROTOCOL_STAGE_8'),
 (0.1260362976949672, 'PROTOCOL_STAGE_9'),
 (0.0, 'PROTOCOL_STAGE_10'),
 (3.330792371993496e-05, 'PROTOCOL_STAGE_11'),
 (1.418607696547133, 'UDDER_PARTS_AFFECTED_STAGE_1'),
 (0.0, 'UDDER_PARTS_CURED_STAGE_1'),
 (4.188040604853257, 'UDDER_PARTS_AFFECTED_STAGE_2

In [8]:
from catboost import cv


In [9]:
params = {"n_estimators": 1000,
        "loss_function":'RMSE',
        "learning_rate": 0.4,
        "depth": 3,
        "task_type": 'CPU',
        "random_state": 1,
        "verbose": False}

In [10]:
cv(pool_all, params, fold_count=5)

Training on fold [0/5]

bestTest = 13.30997907
bestIteration = 995

Training on fold [1/5]

bestTest = 14.39507327
bestIteration = 999

Training on fold [2/5]

bestTest = 14.40598198
bestIteration = 999

Training on fold [3/5]

bestTest = 14.24021297
bestIteration = 938

Training on fold [4/5]

bestTest = 14.48057334
bestIteration = 793



,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,26.101624,0.723550,26.032852,0.165404
1,1,21.334073,0.723694,21.334214,0.191701
2,2,19.040149,0.758310,18.997766,0.121198
3,3,17.922196,0.716815,17.874395,0.119086
4,4,17.429234,0.718887,17.348154,0.133250
...,...,...,...,...,...
995,995,14.188322,0.494156,9.514728,0.150518
996,996,14.188728,0.492481,9.512285,0.150323
997,997,14.188530,0.492302,9.510016,0.152261
998,998,14.185953,0.490909,9.507920,0.152608


In [45]:
data = pd.DataFrame(columns = X.columns)
    

for i in range(1, 9):
    X['PROTOCOL_STAGE_1'] = i
    data = pd.concat([data, X[:1]])
    dict(zip(cb.predict(data), list(range(1, 9))))

dict(zip(cb.predict(data), list(range(1, 9))))

{19.656889166361704: 1,
 30.081643406349556: 2,
 20.568979899292955: 3,
 15.938347369225093: 4,
 20.2086474428189: 5,
 18.111311420919304: 6,
 37.00902253487548: 7,
 40.05090053574107: 8}